In [7]:
search_for = 96
start_from = 0
threads = 1
thread = 0
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.002243518829345703
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 630
all cases: 400479387
type: [1, 1, 1, 1, 96] 96
cases of this type: 84934656
10000 34.26839258542856
20000 38.69011444441097
30000 67.2608374468161
40000 183.4313279642702
50000 172.73350579354937
60000 174.5038518921861
70000 171.53338720678974
80000 191.4714927603043
90000 204.70433470702608
100000 197.36983577109532
110000 212.20321183591005
120000 233.31149133736528
130000 255.10356450388892
140000 244.38126480179346
150000 254.06215158424754
160000 239.35919344936502
170000 248.97282245183965
180000 240.81436551009494
190000 254.8243553858783
200000 249.23520922481384
210000 235.45121725655176
220000 264.80806113183826
230000 289.7069728947246
240000 262.1201493251363
250000 256.16474131356415
260000 245.4569563335605
270000 247.01689239176116
280000 242.54990341382307
290000 247.6889027261299
300000 231.86587536802008
310000 239.6258

3050000 223.09556029857458
3060000 202.0847129970111
3070000 233.1003853977478
3080000 249.3895088836847
3090000 35.10376948524272
3100000 35.21222701460006
3110000 35.12136135620478
3120000 34.5655741764552
3130000 180.28784017162212
3140000 273.5188411021169
3150000 228.21256435859817
3160000 226.40303553299393
3170000 217.4798320617754
3180000 212.41840428190952
3190000 220.59818249926226
3200000 219.437484320063
3210000 220.57172012333055
3220000 212.79150456999713
3230000 217.03786121636497
3240000 202.27647026692446
3250000 208.8625472759475
3260000 209.18052621191413
3270000 216.21172854080595
3280000 216.95130896269177
3290000 213.06175180148398
3300000 214.61044998343462
3310000 229.85236649799302
3320000 238.54351687914053
3330000 230.99478892287036
3340000 232.96839999624322
3350000 231.51084055723078
3360000 219.95333377769882
3370000 223.00096630024348
3380000 221.30028966872518
3390000 216.58781919606113
3400000 220.71600234106663
3410000 217.30921504395263
3420000 218.44

6130000 84.22193457689062
6140000 85.04332981433411
6150000 85.17942032269985
6160000 84.78859713402527
6170000 179.39411677900034
6180000 240.2160262527296
6190000 237.75007601763156
6200000 247.69049976088624
6210000 237.85568750158396
6220000 218.04343124622358
6230000 220.76029179494927
6240000 252.4016686657708
6250000 68.92591721333856
6260000 69.15866151835841
6270000 70.17783926954424
6280000 69.43711930667307
6290000 161.71243470797236
6300000 245.0083629446964
6310000 227.35193285158118
6320000 223.76963088889113
6330000 229.60438607072425
6340000 218.47851660775078
6350000 217.86909739559601
6360000 209.90572467146893
6370000 257.71459625483794
6380000 212.29254315063372
6390000 214.52992631446932
6400000 218.16958585751811
6410000 207.2251575523657
6420000 219.53426523944592
6430000 214.79208018080564
6440000 214.4032891962322
6450000 222.49319973027386
6460000 218.17817274198848
6470000 218.61595265087615
6480000 195.14961656383068
6490000 216.13433945764373
6500000 215.67

9220000 202.94919684132586
9230000 195.0970306901905
9240000 191.95714787178082
9250000 194.68875726543803
9260000 212.1889551845312
9270000 249.93076773354653
9280000 56.04703091891564
9290000 34.90961964238454
9300000 34.87517536548851
9310000 36.2654505022319
9320000 97.42342693468343
9330000 263.9208953609056
9340000 208.07894940441003
9350000 185.99552992231958
9360000 187.95034054830458
9370000 198.78005526963827
9380000 207.85633786775145
9390000 205.17065284288444
9400000 206.11934084521084
9410000 207.2875327321532
9420000 201.3593514136945
9430000 211.14487369532384
9440000 209.51467191303345
9450000 207.62448149642915
9460000 222.14320057866507
9470000 230.5029585675144
9480000 235.2230241728241
9490000 220.94654672906435
9500000 218.7746317177793
9510000 220.52813106636498
9520000 218.829579688895
9530000 216.04192969943392
9540000 209.57958784309815
9550000 214.49135123391832
9560000 248.0951450796632
9570000 253.09362634802991
9580000 223.52900396740063
9590000 235.157635

12220000 226.28880399552924
12230000 227.1175675636698
12240000 237.378501914509
12250000 206.8169302244957
12260000 84.04212247451026
12270000 84.96285483238101
12280000 84.85741718023075
12290000 82.5461906142458
12300000 201.20321612303263
12310000 252.2362271002273
12320000 218.73680952092707
12330000 215.98133941881795
12340000 217.5565114763708
12350000 220.859385770387
12360000 221.31620527315945
12370000 222.76105000392204
12380000 245.88601669885992
12390000 244.26616724691706
12400000 239.99722323594807
12410000 237.67998015408307
12420000 227.50967698485385
12430000 222.91254283715676
12440000 228.43403918318623
12450000 220.26506922587828
12460000 221.66843080261876
12470000 233.4411707145072
12480000 207.0723444718758
12490000 248.24338482757886
12500000 212.13079361507218
12510000 68.2106063237209
12520000 67.96767838329019
12530000 67.87267417426737
12540000 68.27397633935443
12550000 218.79755557856322
12560000 251.57768624232733
12570000 222.2694611432917
12580000 235.

...

394950000 0.3474437082500259
394960000 0.35671775405565237
394970000 0.2922370119992362
394980000 0.17359148655062914
394990000 0.17330178779302038
395000000 0.17318472794257933
395010000 0.17264379390643042
395020000 0.16887913729180892
395030000 0.1729189896286527
395040000 0.326825524297754
395050000 0.3542725323353873
395060000 0.3415454039839837
395070000 0.283714867440939
395080000 0.303293758867032
395090000 0.3415079790964391
best so far: 0
type: [2, 3, 1, 2, 8] 96
cases of this type: 98304
395100000 0.2621798660196464
395110000 0.2596869718997876
395120000 0.27466148924573924
395130000 0.21068805612415076
395140000 0.16906836886079446
395150000 0.16869024095371696
395160000 0.16857656059330703
395170000 0.16827322363774644
395180000 0.1998368536039922
395190000 0.22402722741951542
best so far: 0
type: [2, 3, 1, 4, 4] 96
cases of this type: 49152
395200000 0.2004623094578849
395210000 0.19774441634494064
395220000 0.1968264009825587
395230000 0.19248849943902097
best so far: 0


396300000 0.1317840551636815
396310000 0.13122737293618256
396320000 0.1311319031204515
396330000 0.1307898424463868
396340000 0.130168205089536
396350000 0.1300496078714331
396360000 0.12977357462797562
396370000 0.1294349203885595
396380000 0.4362359698937468
396390000 0.4961694262717565
396400000 0.34659554127386544
396410000 0.6147539073663619
396420000 0.4535962786154151
396430000 0.20785118697914148
396440000 0.4727860862499145
396450000 0.4365086631457805
396460000 0.2163040324869421
396470000 0.34156145017121897
396480000 0.47900087457738316
396490000 0.3379219312310683
396500000 0.3092266970000929
396510000 0.545590533028245
396520000 0.41047843873267037
396530000 0.30295993137685456
396540000 0.4908240607707302
396550000 0.4128084694106315
396560000 0.3470065265510612
396570000 0.3698117843440175
396580000 0.47485145947974927
396590000 0.3009729300300744
396600000 0.3352380732430816
396610000 0.4706811271837288
396620000 0.3444404044809871
396630000 0.3070587513276338
3966400

398600000 0.10843167738545603
398610000 0.10831931419640779
best so far: 0
type: [3, 2, 1, 2, 8] 96
cases of this type: 65536
398620000 0.09129688991231388
398630000 0.0820374635197189
398640000 0.08338125445461272
398650000 0.05784848657257027
398660000 0.05765458138748673
398670000 0.05741982338458299
best so far: 0
type: [3, 2, 1, 4, 4] 96
cases of this type: 32768
398680000 0.076640472400407
398690000 0.06814988891322084
398700000 0.06470497903277476
398710000 0.0639207652327617
best so far: 0
type: [3, 2, 1, 8, 2] 96
cases of this type: 16384
398720000 0.05250685306202041
best so far: 0
type: [3, 2, 1, 16, 1] 96
cases of this type: 8192
398730000 0.048806855307042596
best so far: 0
type: [3, 2, 2, 1, 8] 96
cases of this type: 32768
398740000 0.05850095328266091
398750000 0.06717528236009014
398760000 0.054295072085022926
398770000 0.06580913144108322
best so far: 0
type: [3, 2, 2, 2, 4] 96
cases of this type: 16384
398780000 0.0594378029145201
best so far: 0
type: [3, 2, 2, 4, 2] 

best so far: 0
type: [4, 1, 8, 3, 1] 96
cases of this type: 1728
best so far: 0
type: [4, 1, 12, 1, 2] 96
cases of this type: 2304
400020000 0.013736541956961154
best so far: 0
type: [4, 1, 12, 2, 1] 96
cases of this type: 1152
best so far: 0
type: [4, 1, 24, 1, 1] 96
cases of this type: 576
best so far: 0
type: [4, 2, 1, 1, 12] 96
cases of this type: 41472
400030000 0.020880620905405944
400040000 0.017856828742033905
400050000 0.013581826238811018
400060000 0.018596517217775187
best so far: 0
type: [4, 2, 1, 2, 6] 96
cases of this type: 20736
400070000 0.017145728769765962
400080000 0.012764788320024808
best so far: 0
type: [4, 2, 1, 3, 4] 96
cases of this type: 13824
400090000 0.013179803902864459
best so far: 0
type: [4, 2, 1, 4, 3] 96
cases of this type: 10368
400100000 0.013383485349966421
best so far: 0
type: [4, 2, 1, 6, 2] 96
cases of this type: 6912
400110000 0.012556706519007683
best so far: 0
type: [4, 2, 1, 12, 1] 96
cases of this type: 3456
best so far: 0
type: [4, 2, 2, 1

best so far: 0
type: [8, 2, 1, 2, 3] 96
cases of this type: 1296
best so far: 0
type: [8, 2, 1, 3, 2] 96
cases of this type: 864
best so far: 0
type: [8, 2, 1, 6, 1] 96
cases of this type: 432
400460000 0.0005908043268389173
best so far: 0
type: [8, 2, 2, 1, 3] 96
cases of this type: 648
best so far: 0
type: [8, 2, 2, 3, 1] 96
cases of this type: 216
best so far: 0
type: [8, 2, 3, 1, 2] 96
cases of this type: 288
best so far: 0
type: [8, 2, 3, 2, 1] 96
cases of this type: 144
best so far: 0
type: [8, 2, 6, 1, 1] 96
cases of this type: 72
best so far: 0
type: [8, 3, 1, 1, 4] 96
cases of this type: 768
best so far: 0
type: [8, 3, 1, 2, 2] 96
cases of this type: 384
best so far: 0
type: [8, 3, 1, 4, 1] 96
cases of this type: 192
best so far: 0
type: [8, 3, 2, 1, 2] 96
cases of this type: 192
best so far: 0
type: [8, 3, 2, 2, 1] 96
cases of this type: 96
best so far: 0
type: [8, 3, 4, 1, 1] 96
cases of this type: 48
best so far: 0
type: [8, 4, 1, 1, 3] 96
cases of this type: 324
best so fa